In [1]:
import warnings
warnings.filterwarnings(action='ignore')


In [2]:
import pandas as pd
import geopandas as gpd
import requests

import shapely
from shapely.geometry import Polygon, Point
from shapely.ops import cascaded_union
import shapely.speedups

shapely.speedups.enable()

import folium
from folium.plugins import MeasureControl

import plotly.express as px
import time

pd.set_option('display.max_rows', 500)

In May 2021, Nepal had a limited number of COVID-19 test labs, with only 69 operational facilities. By November 2021, the number increased to 89. This study aimed to assess the impact of the opening of COVID-19 test labs on the physical accessibility of vulnerable populations in Nepal. By combining public data from the Ministry of Health and Population (MoHP) with global datasets on population, healthcare centers, travel time, and relative wealth index, this research aimed to provide critical insights on optimizing the location of testing facilities using open data and optimization models, with a focus on confronting unprecedented pandemics such as COVID-19.

In [ ]:
existing_labs = pd.read_csv('Data/laboratory_geocoded.csv')
existing_labs = existing_labs[['Unnamed: 0','lng','lat','Laboratory']]
existing_labs.columns = ['index','LONG','LAT','L_NAME']
current_hospitals = existing_labs[['index','LONG','LAT','L_NAME']].drop_duplicates()
current_hospitals.columns = ['Hosp_ID','Longitude','Latitude','L_NAME']

df_lab_open_dates = pd.read_excel('Data/tests_per_lab.xlsx')[['Date','Laboratory','Province Name']]
df_open_dates = df_lab_open_dates.groupby(['Laboratory','Province Name'])['Date'].min().reset_index()
df_open_dates.columns = ['L_NAME','Province','Date Open']

In [ ]:
existing_labs = pd.merge(current_hospitals,df_open_dates,on='L_NAME')

def convert_point(df):
    lat = df['Latitude']
    lon = df['Longitude']
    return(Point(lon,lat))

existing_labs['geometry'] = existing_labs[['Latitude','Longitude']].apply(convert_point,axis=1)

existing_labs.head(2)

In [ ]:
len(existing_labs)

In [33]:
%%time
population = pd.read_csv(r'Data/ppp_NPL_2020_1km_Aggregated_UNadj.csv').reset_index()
population.columns = ['ID','xcoord','ycoord','household_count']
population['xcoord'] = population['xcoord'].round(2)
population['ycoord'] = population['ycoord'].round(2)

population = population.groupby(['xcoord','ycoord'])['household_count'].sum().reset_index().reset_index()
population['household_count'] = population['household_count'].round()
population.columns = ['ID','xcoord','ycoord','population']

def convert_Point(population):
    return Point(population['xcoord'],population['ycoord'])
population['geometry'] = population[['ycoord','xcoord']].apply(convert_Point,axis=1)
population = gpd.GeoDataFrame(population)

CPU times: user 8.51 s, sys: 480 ms, total: 8.99 s
Wall time: 9.25 s


In [34]:
print('Total Population:',round(population['population'].sum()/1000000,2),'million')

Total Population: 29.14 million


In [24]:
districts_nepal = gpd.read_file('Data/shapefile_nepal_districts.geojson')

In [22]:
access_token = "pk.eyJ1IjoicGFydmF0aHlrcmlzaG5hbmthYnciLCJhIjoiY2xqZXlna2VtMDFyOTNrbThqcnE2ZWx2ZiJ9.fwUyC3lZIYnj9hoEwgywHQ"


In [23]:
def get_isochrone(df,minutes_list,access_token,mode):
    longitude = df['Longitude']
    latitude = df['Latitude']
    query = """https://api.mapbox.com/isochrone/v1/mapbox/"""
    query = query+mode+'/'
    query = query+str(longitude)+','+str(latitude)+'?'
    query = query+'contours_minutes='+minutes_list
    query = query+'&polygons=true&access_token='
    query = query+access_token
    req_return = (requests.get(query).json())
    
    if('code' in req_return):
        if (req_return['code']=='NoSegment'):
            print('No Segment')
        else:
            print(req_return)     
    else:
        #print(req_return)     
        return(req_return['features'])

In [25]:
def get_pop_count(x):
    pop_count = population[population['ID'].isin(x)]['population'].sum()
    return pop_count

In [ ]:
%%time

existing_labs['isochrone_60min_driving'] = existing_labs[['Longitude','Latitude']].apply(get_isochrone,
                                                                                   minutes_list="60",
                                                                                   access_token=access_token,
                                                                                   mode='driving',
                                                                                   axis=1)

existing_labs['isochrone_30min_driving'] = existing_labs[['Longitude','Latitude']].apply(get_isochrone,
                                                                                   minutes_list="30",
                                                                                   access_token=access_token,
                                                                                   mode='driving',
                                                                                   axis=1)

existing_labs['isochrone_60min_walking'] = existing_labs[['Longitude','Latitude']].apply(get_isochrone,
                                                                                   minutes_list="60",
                                                                                   access_token=access_token,
                                                                                   mode='walking',
                                                                                   axis=1)

existing_labs['isochrone_30min_walking'] = existing_labs[['Longitude','Latitude']].apply(get_isochrone,
                                                                                   minutes_list="30",
                                                                                   access_token=access_token,
                                                                                   mode='walking',
                                                                                   axis=1)

In [ ]:
%%time

existing_labs['60min_driving'] = existing_labs['isochrone_60min_driving'].apply(lambda x: x[0]['geometry'])
existing_labs['60min_driving'] = existing_labs['60min_driving'].apply(lambda x:Polygon(x['coordinates'][0]))

existing_labs['30min_driving'] = existing_labs['isochrone_30min_driving'].apply(lambda x: x[0]['geometry'])
existing_labs['30min_driving'] = existing_labs['30min_driving'].apply(lambda x:Polygon(x['coordinates'][0]))

existing_labs['60min_walking'] = existing_labs['isochrone_60min_walking'].apply(lambda x: x[0]['geometry'])
existing_labs['60min_walking'] = existing_labs['60min_walking'].apply(lambda x:Polygon(x['coordinates'][0]))

existing_labs['30min_walking'] = existing_labs['isochrone_30min_walking'].apply(lambda x: x[0]['geometry'])
existing_labs['30min_walking'] = existing_labs['30min_walking'].apply(lambda x:Polygon(x['coordinates'][0]))


In [ ]:
def get_population_within_vector(vector_polygon,vector_layer):
    pip_mask = vector_layer.within(vector_polygon)
    pip_data = vector_layer.loc[pip_mask]
    return(list(pip_data['ID'].unique()))

In [ ]:
%%time

existing_labs['ID_60min_driving'] = existing_labs['60min_driving'].apply(get_population_within_vector,vector_layer=population)
existing_labs['ID_30min_driving'] = existing_labs['30min_driving'].apply(get_population_within_vector,vector_layer=population)
existing_labs['ID_60min_walking'] = existing_labs['60min_walking'].apply(get_population_within_vector,vector_layer=population)
existing_labs['ID_30min_walking'] = existing_labs['30min_walking'].apply(get_population_within_vector,vector_layer=population)


In [ ]:
selected_hosp = existing_labs['L_NAME'].unique()[4]
selected_hosp

In [ ]:
start_coords = (28.4939,84.1240)
folium_map = folium.Map(location=start_coords, zoom_start=5)

test_ids = existing_labs[existing_labs['L_NAME']==selected_hosp]


for i in range(0,len(test_ids)):
    folium.Marker([test_ids.iloc[i]['Latitude'], test_ids.iloc[i]['Longitude']],
                        color='blue',popup=test_ids.iloc[i]['L_NAME']).add_to(folium_map)
    
    geo_j = folium.GeoJson(data=test_ids.iloc[i]['60min_driving'],style_function=lambda x:{'color': 'red'})
    folium.Popup(test_ids.iloc[i]['L_NAME']).add_to(geo_j)
    geo_j.add_to(folium_map)
    
    geo_j = folium.GeoJson(data=test_ids.iloc[i]['30min_driving'],style_function=lambda x:{'color': 'cyan'})
    folium.Popup(test_ids.iloc[i]['L_NAME']).add_to(geo_j)
    geo_j.add_to(folium_map)
    
    geo_j = folium.GeoJson(data=test_ids.iloc[i]['60min_walking'],style_function=lambda x:{'color': 'blue'})
    folium.Popup(test_ids.iloc[i]['L_NAME']).add_to(geo_j)
    geo_j.add_to(folium_map)
    
    geo_j = folium.GeoJson(data=test_ids.iloc[i]['30min_walking'],style_function=lambda x:{'color': 'green'})
    folium.Popup(test_ids.iloc[i]['L_NAME']).add_to(geo_j)
    geo_j.add_to(folium_map)
    
folium_map

## Calculating percentages for all open test centres.

In [ ]:
list_pop_ids = list(existing_labs['ID_30min_walking'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population[population['ID'].isin(pop_with_access)]['population'].sum()*100/population['population'].sum()).round()

In [ ]:
list_pop_ids = list(existing_labs['ID_60min_walking'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population[population['ID'].isin(pop_with_access)]['population'].sum()*100/population['population'].sum()).round()

In [ ]:
list_pop_ids = list(existing_labs['ID_30min_driving'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population[population['ID'].isin(pop_with_access)]['population'].sum()*100/population['population'].sum()).round()

In [ ]:
list_pop_ids = list(existing_labs['ID_60min_driving'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population[population['ID'].isin(pop_with_access)]['population'].sum()*100/population['population'].sum()).round()

## Calculating percentages for test centres open in May 2021

In [ ]:
initial_labs = existing_labs[existing_labs['Date Open']=='2021-05-01']

In [ ]:
list_pop_ids = list(initial_labs['ID_30min_walking'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population[population['ID'].isin(pop_with_access)]['population'].sum()*100/population['population'].sum()).round()


In [ ]:
list_pop_ids = list(initial_labs['ID_60min_walking'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population[population['ID'].isin(pop_with_access)]['population'].sum()*100/population['population'].sum()).round()

In [ ]:
list_pop_ids = list(initial_labs['ID_30min_driving'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population[population['ID'].isin(pop_with_access)]['population'].sum()*100/population['population'].sum()).round()

In [ ]:
list_pop_ids = list(initial_labs['ID_60min_driving'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population[population['ID'].isin(pop_with_access)]['population'].sum()*100/population['population'].sum()).round()

### 60 min driving shows the highest increase in percentage from 55 to 62% 

In [ ]:
df_relative_wealth_index = pd.read_csv('Data/npl_relative_wealth_index.csv')
def convert_Point(population):
    return Point(population['longitude'],population['latitude'])
df_relative_wealth_index['geometry'] = df_relative_wealth_index[['latitude','longitude']].apply(convert_Point,axis=1)
df_relative_wealth_index = gpd.GeoDataFrame(df_relative_wealth_index)
rwi_districts = gpd.sjoin(df_relative_wealth_index, districts_nepal, how="left", op="within")
rwi_districts = rwi_districts.groupby(['DISTRICT','Province'])['rwi'].mean().reset_index()
rwi_districts['rwi'] = rwi_districts['rwi'].round(2)
rwi_districts.columns = ['District','Province','Relative Wealth Index']

In [ ]:
list_pop_ids = list(initial_labs['ID_60min_driving'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]

district_analysis = population[population['ID'].isin(list_pop_ids)][['population','DISTRICT','Province']]
district_analysis.columns = ['Pop with Access','District','Province']
dt1 = district_analysis.groupby(['District','Province'])['Pop with Access'].sum().reset_index()

dt2 = population.groupby(['DISTRICT','Province'])['population'].sum().reset_index()
dt2.columns = ['District','Province','Total Population']

df_pop_access_dist = pd.merge(dt1,dt2,on=['District','Province'])
df_pop_access_dist['%'] = df_pop_access_dist['Pop with Access']*100/df_pop_access_dist['Total Population']
df_pop_access_dist['%'] = df_pop_access_dist['%'].round(2)

df_pop_access_dist = pd.merge(df_pop_access_dist,rwi_districts,on=['District','Province'])

fig = px.scatter(df_pop_access_dist,y='%',x='Relative Wealth Index',color='Province',size='Total Population')
fig.update_xaxes(title='Average Relative Wealth Index of District')
fig.update_yaxes(title='% of Population with Access')
fig.update_layout(plot_bgcolor='white',title='60 Min Driving for all 89 labs')

### Data of potential locations as list of hospitals

In [ ]:
potential_locations = gpd.read_file('Data/grid_nepal_5km.geojson').reset_index()
potential_locations = potential_locations[['index','geometry']]
potential_locations.columns = ['ID','geometry']
potential_locations['Latitude'] = potential_locations['geometry'].apply(lambda x:x.y)
potential_locations['Longitude'] = potential_locations['geometry'].apply(lambda x:x.x)

In [14]:
potential_locations = gpd.read_file('Data/healthsites_facs.geojson')
potential_locations = potential_locations[['amenity','name','geometry']].reset_index()
potential_locations.columns = ['ID','Facility Type','Name','geometry']
potential_locations = potential_locations[~potential_locations['Facility Type'].isin(['pharmacy','dentist'])]
potential_locations['Latitude'] = potential_locations['geometry'].apply(lambda x:x.y)
potential_locations['Longitude'] = potential_locations['geometry'].apply(lambda x:x.x)

In [20]:
potential_locations.head(2)

,ID,Facility Type,Name,geometry,Latitude,Longitude
0,0,hospital,Shrikrishnagandaki Hospital,POINT (83.58827 27.97347),27.973468,83.588272
1,1,hospital,Nepal Cancer Hospital,POINT (85.33890 27.64347),27.643465,85.338902


In [26]:
# Delay function
def delay():
    print('Waiting')
    time.sleep(60)

In [37]:
%%time

# Delay after 300 requests, 60 mins driving
delay()
counter = 0
for index, row in potential_locations.iterrows():
    if counter == 300:
        delay()
        counter = 0
    # Make the isochrone request
    isochrone_result = get_isochrone(row, "60", access_token, "driving")
    potential_locations.at[index, 'isochrone_60min_driving'] = isochrone_result
    counter += 1

Waiting
Waiting
Waiting
Waiting
Waiting
Waiting


In [38]:
%%time

delay()
# Delay after 300 requests, 30 mins driving
counter = 0
for index, row in potential_locations.iterrows():
    if counter == 300:
        delay()
        counter = 0
    # Make the isochrone request
    isochrone_result = get_isochrone(row, "30", access_token, "driving")
    potential_locations.at[index, 'isochrone_30min_driving'] = isochrone_result
    counter += 1

Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
CPU times: user 27.2 s, sys: 2.61 s, total: 29.8 s
Wall time: 10min


In [50]:
%%time

delay()
# Delay after 300 requests, 60 mins walking
counter = 0
for index, row in potential_locations.iterrows():
    if counter == 300:
        delay()
        counter = 0
    # Make the isochrone request
    isochrone_result = get_isochrone(row, "60", access_token, "walking")
    potential_locations.at[index, 'isochrone_60min_walking'] = isochrone_result
    counter += 1


Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
CPU times: user 22.7 s, sys: 2.56 s, total: 25.3 s
Wall time: 7min 42s


In [27]:
%%time

delay()
# Delay after 300 requests, 30 mins walking
counter = 0
for index, row in potential_locations.iterrows():
    if counter == 300:
        delay()
        counter = 0
    # Make the isochrone request
    isochrone_result = get_isochrone(row, "30", access_token, "walking")
    potential_locations.at[index, 'isochrone_30min_walking'] = isochrone_result
    counter += 1


Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
Waiting
CPU times: user 30.4 s, sys: 3.73 s, total: 34.1 s
Wall time: 11min 59s


In [40]:
def get_geometry(x):
    if(x!=None):
        if(len(x)==1):
            return(x[0]['geometry'])
        else:
            return(x['geometry'])
    else:
        return None
        
def get_cordinates(x):
    if(x!=None):
        if(x['coordinates']!=''):
            return Polygon(x['coordinates'][0])
        else:
            return None
    else:
        return None


In [51]:
%%time

potential_locations['60min_driving'] = potential_locations['isochrone_60min_driving'].apply(get_geometry)
potential_locations['60min_driving'] = potential_locations['60min_driving'].apply(get_cordinates)

potential_locations['30min_driving'] = potential_locations['isochrone_30min_driving'].apply(get_geometry)
potential_locations['30min_driving'] = potential_locations['30min_driving'].apply(get_cordinates)

potential_locations['60min_walking'] = potential_locations['isochrone_60min_walking'].apply(get_geometry)
potential_locations['60min_walking'] = potential_locations['60min_walking'].apply(get_cordinates)
                                                                                  
potential_locations['30min_walking'] = potential_locations['isochrone_30min_walking'].apply(get_geometry)
potential_locations['30min_walking'] = potential_locations['30min_walking'].apply(get_cordinates)

CPU times: user 304 ms, sys: 56.8 ms, total: 361 ms
Wall time: 356 ms


In [52]:
potential_locations = potential_locations.dropna()

In [53]:
def get_population_within_vector(vector_polygon,vector_layer):
    pip_mask = vector_layer.within(vector_polygon)
    pip_data = vector_layer.loc[pip_mask]
    return(list(pip_data['ID'].unique()))

In [55]:
%%time

potential_locations['ID_60min_driving'] = potential_locations['60min_driving'].apply(get_population_within_vector,vector_layer=population)
potential_locations['ID_30min_driving'] = potential_locations['30min_driving'].apply(get_population_within_vector,vector_layer=population)
potential_locations['ID_60min_walking'] = potential_locations['60min_walking'].apply(get_population_within_vector,vector_layer=population)
potential_locations['ID_30min_walking'] = potential_locations['30min_walking'].apply(get_population_within_vector,vector_layer=population)


CPU times: user 6min 13s, sys: 3.17 s, total: 6min 16s
Wall time: 6min 21s


In [56]:
list_pop_ids = list(potential_locations['ID_30min_walking'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population[population['ID'].isin(pop_with_access)]['population'].sum()*100/population['population'].sum()).round()

24.0

In [57]:
list_pop_ids = list(potential_locations['ID_60min_walking'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population[population['ID'].isin(pop_with_access)]['population'].sum()*100/population['population'].sum()).round()

32.0

In [58]:
list_pop_ids = list(potential_locations['ID_30min_driving'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population[population['ID'].isin(pop_with_access)]['population'].sum()*100/population['population'].sum()).round()

53.0

In [59]:
list_pop_ids = list(potential_locations['ID_60min_driving'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]
pop_with_access = list(set(list_pop_ids))

(population[population['ID'].isin(pop_with_access)]['population'].sum()*100/population['population'].sum()).round()

80.0

In [ ]:
list_pop_ids = list(potential_locations['ID_30min_walking'].values)
list_pop_ids = [item for sublist in list_pop_ids for item in sublist]

district_analysis = population[population['ID'].isin(list_pop_ids)][['population','DISTRICT','Province']]
district_analysis.columns = ['Pop with Access','District','Province']
dt1 = district_analysis.groupby(['District','Province'])['Pop with Access'].sum().reset_index()

dt2 = population.groupby(['DISTRICT','Province'])['population'].sum().reset_index()
dt2.columns = ['District','Province','Total Population']

df_pop_access_dist = pd.merge(dt1,dt2,on=['District','Province'])
df_pop_access_dist['%'] = df_pop_access_dist['Pop with Access']*100/df_pop_access_dist['Total Population']
df_pop_access_dist['%'] = df_pop_access_dist['%'].round(2)

df_pop_access_dist = pd.merge(df_pop_access_dist,rwi_districts,on=['District','Province'])

fig = px.scatter(df_pop_access_dist,y='%',x='Relative Wealth Index',color='Province',size='Total Population')
fig.update_xaxes(title='Average Relative Wealth Index of District')
fig.update_yaxes(title='% of Population with Access')
fig.update_layout(plot_bgcolor='white',title='30 Min Walking - 10 km grid Access')

In [ ]:
existing_labs[['Hosp_ID','L_NAME','Province',
               'Date Open','geometry',
               'ID_60min_driving','ID_30min_driving',
               'ID_60min_walking','ID_30min_walking']].to_pickle('Data/Results_Analytics/existing_labs.pkl')

In [60]:
potential_locations[['ID','geometry',
               'ID_60min_driving','ID_30min_driving',
               'ID_60min_walking','ID_30min_walking']].to_pickle('Data/Results_Analytics/potential_locs_healthsites.pkl')


In [ ]:
population.to_pickle('Data/Results_Analytics/population.pkl')

In [ ]:
existing_labs = pd.read_pickle('Data/Results_Analytics/existing_labs.pkl')
population = pd.read_pickle('Data/Results_Analytics/population.pkl')
potential_locations = pd.read_pickle('Data/Results_Analytics/potential_locs_hospitals.pkl')


In [ ]:
population.head(2)

In [ ]:
df_relative_wealth_index = pd.read_csv('Data/npl_relative_wealth_index.csv')
def convert_Point(population):
    return Point(population['longitude'],population['latitude'])
df_relative_wealth_index['geometry'] = df_relative_wealth_index[['latitude','longitude']].apply(convert_Point,axis=1)
df_relative_wealth_index = gpd.GeoDataFrame(df_relative_wealth_index)


In [69]:
potential_locations.head(2)

,ID,Facility Type,Name,geometry,Latitude,Longitude,isochrone_30min_walking,30min_walking,ID_30min_walking,isochrone_60min_driving,isochrone_30min_driving,isochrone_60min_walking,60min_driving,30min_driving,60min_walking,ID_60min_driving,ID_30min_driving,ID_60min_walking
0,0,hospital,Shrikrishnagandaki Hospital,POINT (83.58827 27.97347),27.973468,83.588272,"{'properties': {'fill': '#bf4040', 'fillOpacit...","POLYGON ((83.580272 27.980299, 83.579609 27.98...","[66386, 66387, 66558, 66729]","{'properties': {'fill': '#bf4040', 'fillOpacit...","{'properties': {'fill': '#bf4040', 'fillOpacit...","[{'properties': {'fill': '#bf4040', 'fillOpaci...","POLYGON ((83.588272 28.094434, 83.585589 28.09...","POLYGON ((83.578272 28.032474, 83.574587 28.03...","POLYGON ((83.580272 27.980357, 83.578372 27.98...","[62935, 62939, 63131, 63137, 63138, 63141, 633...","[64630, 64810, 65166, 65518, 65519, 65692, 656...","[66213, 66386, 66387, 66558, 66729]"
1,1,hospital,Nepal Cancer Hospital,POINT (85.33890 27.64347),27.643465,85.338902,"[{'properties': {'fill': '#bf4040', 'fillOpaci...","POLYGON ((85.336902 27.663588, 85.335902 27.66...","[95359, 95360, 95361, 95514, 95515, 95516, 955...","[{'properties': {'fill': '#bf4040', 'fillOpaci...","[{'properties': {'fill': '#bf4040', 'fillOpaci...","[{'properties': {'fill': '#bf4040', 'fillOpaci...","POLYGON ((85.334902 27.796509, 85.330902 27.79...","POLYGON ((85.306902 27.725765, 85.304902 27.72...","POLYGON ((85.333902 27.682503, 85.333117 27.68...","[93172, 93175, 93176, 93335, 93338, 93339, 933...","[94279, 94280, 94433, 94434, 94435, 94436, 945...","[95048, 95049, 95050, 95051, 95202, 95203, 952..."


In [121]:
start_coords = (28.4939,84.1240)
folium_map = folium.Map(location=start_coords, zoom_start=5)

for i in range(0,len(potential_locations)):
    
    folium.Marker([potential_locations.iloc[i]['Latitude'], potential_locations.iloc[i]['Longitude']],
                        color='blue',popup=potential_locations.iloc[i]['Name']).add_to(folium_map)
    
    geo_j = folium.GeoJson(data=potential_locations.iloc[i]['60min_driving'],
                           style_function=lambda x:{'color': 'violet'})
    geo_j.add_to(folium_map)


In [122]:
population_visualize = population

quartiles = population_visualize['population'].quantile([0.25, 0.5, 0.75]).tolist()

# Define a function to assign quartile labels
def assign_quartile_label(population):
    if population <= quartiles[0]:
        return 0.25
    elif population <= quartiles[1]:
        return 0.5
    elif population <= quartiles[2]:
        return 0.75
    else:
        return 1

population_visualize['quartile'] = population_visualize['population'].apply(assign_quartile_label)

l = list(potential_locations['ID_60min_driving'].values)
flat_list = [item for sublist in l for item in sublist]

pop_with_access = population_visualize[population_visualize['ID'].isin(flat_list)]
pop_without_access = population_visualize[~population_visualize['ID'].isin(flat_list)]

for i in range(0,len(pop_with_access)):
    folium.CircleMarker([pop_with_access.iloc[i]['ycoord'], pop_with_access.iloc[i]['xcoord']],
                        color='green',fill=True,radius=2, 
                        opacity=pop_with_access.iloc[i]['quartile']).add_to(folium_map)
    
for i in range(0,len(pop_without_access)):
    folium.CircleMarker([pop_without_access.iloc[i]['ycoord'], pop_without_access.iloc[i]['xcoord']],
                        color='red',fill=True,radius=2, 
                        opacity=pop_without_access.iloc[i]['quartile']).add_to(folium_map)
    

In [123]:
len(pop_with_access)

37785

In [124]:
folium_map.save('map_60min_driving.html')

In [106]:
population = pd.read_pickle('Data/Results_Analytics/population.pkl')


In [110]:
df_relative_wealth_index = pd.read_csv('Data/npl_relative_wealth_index.csv')
def convert_Point(population):
    return Point(population['longitude'],population['latitude'])
df_relative_wealth_index['geometry'] = df_relative_wealth_index[['latitude','longitude']].apply(convert_Point,axis=1)
df_relative_wealth_index = gpd.GeoDataFrame(df_relative_wealth_index)


In [112]:
population.head(2)

,ID,xcoord,ycoord,population,geometry,index_right,STATE_CODE,DISTRICT,GaPa_NaPa,Type_GN,Province
0,0,80.06,28.91,719.0,POINT (80.06000 28.91000),NaN,NaN,NaN,NaN,NaN,NaN
1,1,80.06,28.92,498.0,POINT (80.06000 28.92000),NaN,NaN,NaN,NaN,NaN,NaN


In [113]:
df_relative_wealth_index.head(2)

,latitude,longitude,rwi,error,geometry
0,29.602118,81.331787,-0.046,0.425,POINT (81.33179 29.60212)
1,27.712710,83.199463,0.061,0.465,POINT (83.19946 27.71271)


In [114]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points

# Assuming your GeoDataFrames are named 'population' and 'rwi'
# Create a copy of the 'population' GeoDataFrame to store the result
population_with_rwi = population.copy()

# Iterate over each row in the 'population' GeoDataFrame
for idx, row in population.iterrows():
    point = row['geometry']
    nearest = nearest_points(point, df_relative_wealth_index.unary_union)[1]  # Get the second point (nearest) from the pair
    
    # Find the corresponding 'rwi' value based on the nearest point
    rwi_value = df_relative_wealth_index[df_relative_wealth_index.geometry == nearest]['rwi'].values[0]
    
    # Update the 'rwi' column in the 'population_with_rwi' GeoDataFrame
    population_with_rwi.loc[idx, 'rwi'] = rwi_value


KeyboardInterrupt: 

In [ ]:
population_with_rwi.head(2)